# Aadhaar Enrolment Dataset: District Name Cleaning
## Cleaning duplicate and inconsistent district names in Indian Aadhaar enrolment data

This notebook standardizes district names while respecting state-district pairs and identifies potential duplicates for manual review.

In [7]:
# Import Required Libraries
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

## Section 1: Load and Explore the Dataset

In [3]:
# Load all CSV files and concatenate them into a single dataframe
data_dir = r'e:\MY project\adhar first project'
csv_files = [
    'api_data_aadhar_enrolment_0_500000.csv',
    'api_data_aadhar_enrolment_500000_1000000.csv',
    'api_data_aadhar_enrolment_1000000_1006029.csv'
]

# Load and combine all CSV files
df_list = []
for file in csv_files:
    file_path = os.path.join(data_dir, file)
    if os.path.exists(file_path):
        df_list.append(pd.read_csv(file_path))
        print(f"Loaded: {file}")

# Concatenate all dataframes
df = pd.concat(df_list, ignore_index=True)
print(f"\nTotal records: {len(df)}")

# Display basic information
print("\n=== Dataset Shape and Info ===")
print(f"Shape: {df.shape}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nFirst few rows:")
print(df.head(10))
print(f"\nDataset Info:")
print(df.info())

NameError: name 'os' is not defined

In [2]:
# Check for initial district issues
print("\n=== Initial District Data Issues ===")
print(f"Total unique districts: {df['district'].nunique()}")
print(f"Missing districts (NaN): {df['district'].isna().sum()}")
print(f"\nSample of unique district values:")
print(df['district'].unique()[:20])

# Check for districts with leading/trailing spaces or multiple spaces
print(f"\nDistricts with leading/trailing spaces:")
spaced_districts = df[df['district'].str.strip() != df['district']]['district'].unique()
print(f"Count: {len(spaced_districts)}")
if len(spaced_districts) > 0:
    print(spaced_districts[:5])


=== Initial District Data Issues ===


NameError: name 'df' is not defined

## Section 2: Replace Invalid District Values

Replace invalid values such as 'Nan', 'None', 'Unknown', '0' with actual NaN values

In [ ]:
# Define invalid district values that should be replaced with NaN
# Using case-insensitive matching
invalid_values = ['nan', 'none', 'unknown', '0', 'na', 'n/a', '-', '']

# Create a copy of the dataframe for processing
df_cleaned = df.copy()

# Replace invalid values (case-insensitive string matching)
mask = df_cleaned['district'].astype(str).str.lower().isin(invalid_values)
print(f"Records with invalid district values: {mask.sum()}")
if mask.sum() > 0:
    print("Invalid values found and being replaced with NaN:")
    print(df_cleaned[mask]['district'].value_counts())

# Replace invalid values with NaN
df_cleaned.loc[mask, 'district'] = np.nan

print(f"\nMissing districts after replacement: {df_cleaned['district'].isna().sum()}")

## Section 3: Standardize District Names

Apply text transformations: strip spaces, remove extra internal spaces, and convert to title case

In [ ]:
# Function to standardize district names
def standardize_district_name(name):
    """
    Standardize a district name by:
    1. Stripping leading/trailing whitespace
    2. Removing extra internal spaces (multiple spaces -> single space)
    3. Converting to title case
    """
    if pd.isna(name):
        return name
    
    # Convert to string if not already
    name = str(name)
    
    # Strip leading/trailing spaces
    name = name.strip()
    
    # Remove extra internal spaces (replace multiple spaces with single space)
    name = ' '.join(name.split())
    
    # Convert to title case
    name = name.title()
    
    return name

# Apply standardization to all district names
print("Applying standardization to district names...")
df_cleaned['district'] = df_cleaned['district'].apply(standardize_district_name)

print("\n=== After Standardization ===")
print(f"Total unique districts: {df_cleaned['district'].nunique()}")
print(f"Missing districts: {df_cleaned['district'].isna().sum()}")
print(f"\nSample of standardized district values:")
print(df_cleaned['district'].unique()[:20])

## Section 4: Apply Manual District Mapping

Handle well-known district name variations using a carefully maintained mapping dictionary.
Only apply mappings for established district renames, not for speculation.

In [ ]:
# Manual mapping dictionary for well-known district name changes
# Only includes official/common renames recognized across India
# Note: Apply mappings per state where applicable

district_mapping = {
    # Karnataka
    ('Karnataka', 'Bangalore'): 'Bengaluru',
    ('Karnataka', 'Bangalore Urban'): 'Bengaluru Urban',
    ('Karnataka', 'Bangalore Rural'): 'Bengaluru Rural',
    
    # West Bengal
    ('West Bengal', 'Calcutta'): 'Kolkata',
    ('West Bengal', '24 Parganas (North)'): 'North Twenty Four Parganas',
    ('West Bengal', '24 Parganas (South)'): 'South Twenty Four Parganas',
    
    # Telangana / Andhra Pradesh
    ('Telangana', 'Hyderabad'): 'Hyderabad',
    ('Andhra Pradesh', 'Hyderabad'): 'Ranga Reddy',
    
    # Bihar
    ('Bihar', 'Darbhanga'): 'Darbhanga',
    
    # Maharashtra - Ahmednagar to Ahmadnagar variants
    ('Maharashtra', 'Ahmednagar'): 'Ahmednagar',
    
    # Uttar Pradesh - Allahabad to Prayagraj
    ('Uttar Pradesh', 'Allahabad'): 'Prayagraj',
}

# Apply the manual mapping
print("=== Applying Manual District Mapping ===")
mapping_applied_count = 0

for (state, old_district), new_district in district_mapping.items():
    # Find rows matching state and standardized old district name
    mask = (df_cleaned['state'] == state) & (df_cleaned['district'] == old_district)
    count = mask.sum()
    
    if count > 0:
        df_cleaned.loc[mask, 'district'] = new_district
        mapping_applied_count += count
        print(f"Mapped: ({state}, {old_district}) -> {new_district} [{count} records]")

print(f"\nTotal records updated via manual mapping: {mapping_applied_count}")
print(f"Total unique districts after mapping: {df_cleaned['district'].nunique()}")

## Section 5: Identify Potential Duplicate Districts by State

Group by (state, district) to identify potential duplicates. 
This allows manual review without automatically dropping rows.

In [ ]:
# Group by (state, district) to identify duplicates
# Important: We're NOT automatically merging, just identifying for manual review

state_district_counts = df_cleaned.groupby(['state', 'district']).size().reset_index(name='record_count')
state_district_counts = state_district_counts.sort_values(['state', 'record_count'], ascending=[True, False])

print("=== State-District Pair Summary ===")
print(f"Total unique (state, district) pairs: {len(state_district_counts)}")
print(f"States in dataset: {df_cleaned['state'].nunique()}")

# Identify potential duplicates: Similar names within same state
# These could be real variations (Urban/Rural) or actual duplicates
print("\n=== Potential Duplicate Analysis ===")
print("(Looking for similar district names within the same state)")

duplicate_candidates = []

for state in df_cleaned['state'].unique():
    if pd.isna(state):
        continue
    
    state_districts = state_district_counts[state_district_counts['state'] == state]['district'].tolist()
    
    # Look for districts that differ only by minor variations
    # (e.g., suffixes like Urban, Rural, District)
    for i, dist1 in enumerate(state_districts):
        if pd.isna(dist1):
            continue
        
        base_name1 = dist1.replace(' Urban', '').replace(' Rural', '').replace(' District', '').strip()
        
        for dist2 in state_districts[i+1:]:
            if pd.isna(dist2):
                continue
            
            base_name2 = dist2.replace(' Urban', '').replace(' Rural', '').replace(' District', '').strip()
            
            # If base names are identical, these might be variations to review
            if base_name1 == base_name2 and dist1 != dist2:
                count1 = state_district_counts[(state_district_counts['state'] == state) & 
                                               (state_district_counts['district'] == dist1)]['record_count'].values[0]
                count2 = state_district_counts[(state_district_counts['state'] == state) & 
                                               (state_district_counts['district'] == dist2)]['record_count'].values[0]
                duplicate_candidates.append({
                    'state': state,
                    'district_1': dist1,
                    'count_1': count1,
                    'district_2': dist2,
                    'count_2': count2,
                    'note': 'Possible variation (Urban/Rural/District suffix)'
                })

if duplicate_candidates:
    print(f"\nFound {len(duplicate_candidates)} potential duplicate pairs:\n")
    dup_df = pd.DataFrame(duplicate_candidates)
    print(dup_df.to_string(index=False))
else:
    print("\nNo obvious duplicate patterns detected (no same base names with different suffixes)")

# Display the full state-district summary
print("\n=== Complete State-District Summary ===")
print(state_district_counts.to_string(index=False))

## Section 6: Review and Drop Rows with Missing Districts

Display rows with missing districts, then remove them from the cleaned dataset

In [ ]:
# Check for missing districts before dropping
missing_districts_mask = df_cleaned['district'].isna()
missing_count = missing_districts_mask.sum()

print("=== Missing Districts Review ===")
print(f"Total rows with missing districts: {missing_count}")

if missing_count > 0:
    print("\nSample of rows with missing districts:")
    print(df_cleaned[missing_districts_mask].head(10))
    
    print("\nMissing districts by state:")
    print(df_cleaned[missing_districts_mask]['state'].value_counts())

# Drop rows where district is NaN after all cleaning steps
print(f"\n=== Dropping Missing Districts ===")
print(f"Rows before dropping: {len(df_cleaned)}")
df_cleaned = df_cleaned[~missing_districts_mask].reset_index(drop=True)
print(f"Rows after dropping: {len(df_cleaned)}")
print(f"Rows removed: {missing_count}")

# Verify no missing districts remain
remaining_missing = df_cleaned['district'].isna().sum()
print(f"\nRemaining missing districts: {remaining_missing}")

## Section 7: Export Cleaned Data and Generate Report

Save the cleaned dataframe and create a comprehensive report of the cleaning process

In [ ]:
# Save the cleaned dataframe
output_file = os.path.join(data_dir, 'aadhar_enrolment_cleaned.csv')
df_cleaned.to_csv(output_file, index=False)
print(f"✓ Cleaned dataset saved to: {output_file}")

# Generate cleaning report
print("\n" + "="*60)
print("CLEANING REPORT SUMMARY")
print("="*60)

print("\n📊 Dataset Overview:")
print(f"  • Original records: {len(df)}")
print(f"  • Final records: {len(df_cleaned)}")
print(f"  • Records removed: {len(df) - len(df_cleaned)}")

print("\n🔄 Standardization Applied:")
print(f"  • Invalid values replaced: {mask.sum()}")
print(f"  • All districts: Title cased, spaces normalized")

print("\n📋 District-State Pairs:")
print(f"  • Total unique (state, district) pairs: {df_cleaned.groupby(['state', 'district']).size().shape[0]}")
print(f"  • States represented: {df_cleaned['state'].nunique()}")

print("\n✅ Data Quality Verification:")
print(f"  • Missing districts remaining: {df_cleaned['district'].isna().sum()}")
print(f"  • Missing states remaining: {df_cleaned['state'].isna().sum()}")

# Show data types
print("\n📌 Final Data Types:")
print(df_cleaned.dtypes)

print("\n✓ Sample of cleaned data:")
print(df_cleaned.head(15))

In [ ]:
# Export state-district summary for reference
state_district_summary = df_cleaned.groupby(['state', 'district']).agg({
    'date': 'count',
    'age_0_5': ['min', 'max', 'mean'],
    'age_5_17': ['min', 'max', 'mean'],
    'age_18_greater': ['min', 'max', 'mean']
}).round(2)

state_district_summary.columns = ['record_count', 'age_0_5_min', 'age_0_5_max', 'age_0_5_mean',
                                   'age_5_17_min', 'age_5_17_max', 'age_5_17_mean',
                                   'age_18_greater_min', 'age_18_greater_max', 'age_18_greater_mean']

state_district_summary = state_district_summary.reset_index().sort_values(['state', 'district'])

summary_file = os.path.join(data_dir, 'state_district_summary.csv')
state_district_summary.to_csv(summary_file, index=False)
print(f"✓ State-District summary saved to: {summary_file}")

# Export duplicate candidates for manual review (if any)
if duplicate_candidates:
    dup_df.to_csv(os.path.join(data_dir, 'potential_duplicates_for_review.csv'), index=False)
    print(f"✓ Potential duplicates saved for manual review")
else:
    print(f"✓ No obvious duplicates detected")

print("\n" + "="*60)
print("CLEANING COMPLETED SUCCESSFULLY")
print("="*60)
print("\nOutput Files Generated:")
print(f"  1. {output_file}")
print(f"  2. {summary_file}")
if duplicate_candidates:
    print(f"  3. {os.path.join(data_dir, 'potential_duplicates_for_review.csv')}")